In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab6").getOrCreate()

24/09/16 10:55:08 WARN Utils: Your hostname, mahe-HP-Z1-G9-Tower-Desktop-PC resolves to a loopback address: 127.0.1.1; using 172.16.57.86 instead (on interface eno1)
24/09/16 10:55:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 10:55:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/16 10:55:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data = spark.read.option("inferSchema", True).option("header", False).csv("covtype.data")

In [3]:
colnames = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", 
            "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", 
            "Horizontal_Distance_To_Fire_Points"] + \
           [f"Wilderness_Area_{i}" for i in range(4)] + \
           [f"Soil_Type_{i}" for i in range(40)] + \
           ["Cover_Type"]

data = data.toDF(*colnames)

data.head()

24/09/16 10:55:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area_0=1, Wilderness_Area_1=0, Wilderness_Area_2=0, Wilderness_Area_3=0, Soil_Type_0=0, Soil_Type_1=0, Soil_Type_2=0, Soil_Type_3=0, Soil_Type_4=0, Soil_Type_5=0, Soil_Type_6=0, Soil_Type_7=0, Soil_Type_8=0, Soil_Type_9=0, Soil_Type_10=0, Soil_Type_11=0, Soil_Type_12=0, Soil_Type_13=0, Soil_Type_14=0, Soil_Type_15=0, Soil_Type_16=0, Soil_Type_17=0, Soil_Type_18=0, Soil_Type_19=0, Soil_Type_20=0, Soil_Type_21=0, Soil_Type_22=0, Soil_Type_23=0, Soil_Type_24=0, Soil_Type_25=0, Soil_Type_26=0, Soil_Type_27=0, Soil_Type_28=1, Soil_Type_29=0, Soil_Type_30=0, Soil_Type_31=0, Soil_Type_32=0, Soil_Type_33=0, Soil_Type_34=0, Soil_Type_35=0, Soil_Type_36=0, Soil_Type_37=0, Soil_Type_38=0, Soil_Type_39=0, Cover_Type=5)

In [4]:
(train_data, test_data) = data.randomSplit([0.9, 0.1])

In [13]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col,udf
from pyspark.ml.feature import VectorAssembler

def unencode(data):
    cols = ['Wilderness_Area_' + str(i) for i in range(4)]
    assembler = VectorAssembler().setInputCols(cols).setOutputCol("wilderness")
    
    unhot_udf = udf(lambda v: v.toArray().tolist().index(1))
    
    data = assembler.transform(data).drop(*cols) \
    .withColumn("wilderness", unhot_udf(col("wilderness")).cast(IntegerType()))
    
    cols = ['Soil_Type_' + str(i) for i in range(40)]
    assembler = VectorAssembler().setInputCols(cols).setOutputCol("soil")
    
    data = assembler.transform(data).drop(*cols) \
    .withColumn("soil", unhot_udf(col("soil")).cast(IntegerType()))
    
    return data

In [14]:
unenc_train_data = unencode(train_data)
unenc_train_data.printSchema()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Cover_Type: integer (nullable = true)
 |-- wilderness: integer (nullable = true)
 |-- soil: integer (nullable = true)



In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

cols = unenc_train_data.columns
input_cols = [c for c in cols if c!='Cover_Type']

assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
classifier = DecisionTreeClassifier(seed=1234, labelCol="Cover_Type",featuresCol="features",
predictionCol="pred")

pipeline = Pipeline(stages=[assembler, classifier])

paramGrid = ParamGridBuilder().addGrid(classifier.impurity, ["gini", "entropy"]) \
.addGrid(classifier.maxDepth, [1, 20]).addGrid(classifier.maxBins, [40, 300]) \
.addGrid(classifier.minInfoGain, [0.0, 0.05]).build()

multiclassEval = MulticlassClassificationEvaluator().setLabelCol("Cover_Type").setPredictionCol("pred"). \
setMetricName("accuracy")

In [18]:
# predictions = model.transform(train_data)

# predictions.select("Cover_Type", "Prediction", "probability").show(10, truncate = False)

NameError: name 'model' is not defined

In [7]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="Prediction")

acc=evaluator.setMetricName("accuracy").evaluate(predictions)
f1=evaluator.setMetricName("f1").evaluate(predictions)

print(f'Accuracy = {acc}  F1 Score = {f1}')

Accuracy = 0.6959315430159844  F1 Score = 0.6795547404621637


In [8]:
confusion_matrix = predictions.groupBy("Cover_Type").pivot("prediction", range(1,8)).count().na.fill(0.0) \
.orderBy("Cover_Type")

confusion_matrix.show()

+----------+------+------+-----+---+---+---+-----+
|Cover_Type|     1|     2|    3|  4|  5|  6|    7|
+----------+------+------+-----+---+---+---+-----+
|         1|132897| 52200|  237|  0|  0|  0| 5436|
|         2| 55853|190579| 7697| 86|  0|  0|  790|
|         3|     0|  2409|29131|635|  0|  0|    0|
|         4|     0|     0| 1491|984|  0|  0|    0|
|         5|     0|  7871|  717|  0|  0|  0|    0|
|         6|     0|  3176|11904|498|  0|  0|    0|
|         7|  7977|    26|   47|  0|  0|  0|10432|
+----------+------+------+-----+---+---+---+-----+



In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder

assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
classifier = DecisionTreeClassifier(seed=1234, labelCol="Cover_Type",featuresCol="features",
predictionCol="pred")

pipeline = Pipeline(stages=[assembler, classifier])

paramGrid = ParamGridBuilder().addGrid(classifier.impurity, ["gini", "entropy"]) \
.addGrid(classifier.maxDepth, [1, 20]).addGrid(classifier.maxBins, [40, 300]) \
.addGrid(classifier.minInfoGain, [0.0, 0.05]).build()

multiclassEval = MulticlassClassificationEvaluator().setLabelCol("Cover_Type").setPredictionCol("pred"). \
setMetricName("accuracy")

In [10]:
from pyspark.ml.tuning import TrainValidationSplit

validator = TrainValidationSplit(seed=1234,estimator=pipeline,evaluator=multiclassEval,estimatorParamMaps=paramGrid,
trainRatio=0.9)

validator_model = validator.fit(train_data)
best_model = validator_model.bestModel

24/09/16 10:24:10 WARN DAGScheduler: Broadcasting large task binary with size 1083.4 KiB
24/09/16 10:24:11 WARN DAGScheduler: Broadcasting large task binary with size 1471.0 KiB
24/09/16 10:24:13 ERROR Executor: Exception in task 10.0 in stage 111.0 (TID 1540)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.tree.impl.DTStatsAggregator.<init>(DTStatsAggregator.scala:77)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$22(RandomForest.scala:651)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$22$adapted(RandomForest.scala:647)
	at org.apache.spark.ml.tree.impl.RandomForest$$$Lambda$4413/0x0000000801cce8f8.apply(Unknown Source)
	at scala.Array$.tabulate(Array.scala:418)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$21(RandomForest.scala:647)
	at org.apache.spark.ml.tree.impl.RandomForest$$$Lambda$4340/0x0000000801cb7c00.apply(Unknown Source)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RD

24/09/16 10:24:13 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 6.0 in stage 111.0 (TID 1536)
java.lang.NullPointerException: Cannot invoke "org.apache.spark.SparkEnv.blockManager()" because the return value of "org.apache.spark.SparkEnv$.get()" is null
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:146)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.Task.run(Task.scala:144)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/lplab/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  Fil

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
metrics = validator_model.validationMetrics
params = validator_model.getEstimatorParamMaps()

In [ ]:
metrics.sort(reverse=True)
print(metrics[0])

In [ ]:
multiclassEval.evaluate(best_model.transform(test_data))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/lplab/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
